In [1]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os
import sqlite3
import re
import pdfplumber
import pdfplumber
from flask import Flask, request, jsonify
import base64

SCOPES = ["https://www.googleapis.com/auth/gmail.readonly", "https://www.googleapis.com/auth/gmail.compose", "https://www.googleapis.com/auth/gmail.send", "https://www.googleapis.com/auth/gmail.modify"]

def get_unread_emails_with_subject(service, user_id, subject):
    try:
        query = f'is:unread subject:{subject}'
        results = service.users().messages().list(userId=user_id, q=query).execute()
        messages = results.get('messages', [])
        return messages
    except HttpError as error:
        print(f"An error occurred: {error}")
        return []

def get_attachments(service, user_id, msg_id, store_dir):
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id).execute()
        for part in message['payload']['parts']:
            if part['filename']:
                if 'data' in part['body']:
                    data = part['body']['data']
                else:
                    att_id = part['body']['attachmentId']
                    att = service.users().messages().attachments().get(userId=user_id, messageId=msg_id, id=att_id).execute()
                    data = att['data']
                file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
                
                filename = part['filename']
                path = os.path.join(store_dir, filename)
                if os.path.exists(path):
                    base, extension = os.path.splitext(filename)
                    counter = 1
                    while os.path.exists(path):
                        filename = f"{base}_{counter}{extension}"
                        path = os.path.join(store_dir, filename)
                        counter += 1
                
                with open(path, 'wb') as f:
                    f.write(file_data)
                print(f'Attachment {filename} saved to {path}')
    except HttpError as error:
        print(f"An error occurred: {error}")

def mark_as_read(service, user_id, msg_id):
    try:
        service.users().messages().modify(userId=user_id, id=msg_id, body={'removeLabelIds': ['UNREAD']}).execute()
        print(f'Message {msg_id} marked as read.')
    except HttpError as error:
        print(f"An error occurred: {error}")


def get_db_connection():
    conn = sqlite3.connect('job_requirements.db')
    conn.row_factory = sqlite3.Row
    return conn


def calculate_score(job, experience, qualifications, education):
    conn = get_db_connection()
    job_req = conn.execute('SELECT * FROM job_requirements WHERE jobname = ?', (job,)).fetchone()
    conn.close()

    if job_req is None:
        return 0, f"Initiativ + {job}"

    weight_experience = job_req['weight_experience']
    weight_qualifications = job_req['weight_qualifications']
    weight_education = job_req['weight_education']

    # Normalize weights to sum to 1
    total_weight = weight_experience + weight_qualifications + weight_education
    weight_experience /= total_weight
    weight_qualifications /= total_weight
    weight_education /= total_weight

    # Berechnung des Scores
    score = 0

    # Berufserfahrung bewerten
    req_experience = job_req['experience']
    job_experience = 0

    # Regex für die spezifische Joberfahrung
    job_experience_matches = re.findall(rf'(\d+)\s*(Jahre|Monate)?\s*(als|in|Tätigkeit)?\s*{job}', experience, re.IGNORECASE)
    for match in job_experience_matches:
        years_or_months = int(match[0])
        if 'Monate' in match[1]:
            years_or_months /= 12  # Monate in Jahre umrechnen
        job_experience += years_or_months

    # Berechnung des Scores basierend auf dem Verhältnis der Erfahrung
    min_years = int(re.findall(r'\d+', req_experience)[0])
    if job_experience >= min_years:
        # Volle Punkte und zusätzliche Punkte für mehr Erfahrung
        score += weight_experience * 100
        extra_experience = job_experience - min_years
        score += weight_experience * extra_experience * 10  # Beispiel: 10 Punkte pro zusätzliches Jahr
    else:
        # Teilweise Punkte basierend auf dem Verhältnis
        score += weight_experience * (job_experience / min_years) * 100

    # Qualifikationen bewerten
    req_qualifications = job_req['qualifications']
    qualifications_list = re.split(r'[;, ]', qualifications)  # Trennung durch Semikolon, Komma oder Leerzeichen
    req_qualifications_list = re.split(r'[;, ]', req_qualifications)
    qualifications_count = sum(1 for q in qualifications_list if q.strip().lower() in map(str.lower, req_qualifications_list))
    req_qualifications_count = len(req_qualifications_list)
    if qualifications_count >= req_qualifications_count:
        score += weight_qualifications * 100
    else:
        score += weight_qualifications * (qualifications_count / req_qualifications_count) * 100

    # Sprachkenntnisse bewerten
    language_levels = {
        'muttersprache': 100,
        'bilingual': 100,
        'fließend': 80,
        'fluent': 80,
        'berufliche kenntnisse': 60,
        'professional working proficiency': 60,
        'gute kenntnisse': 50,
        'proficient': 50,
        'konversationssicher': 30,
        'conversational': 30,
        'grundkenntnisse': 10,
        'basic': 10,
        'a1': 10,
        'a2': 20,
        'b1': 30,
        'b2': 40,
        'c1': 50,
        'c2': 100
    }

    req_languages = job_req['languages']
    req_languages_list = re.split(r'[;, ]', req_languages)
    language_score = 0

    for lang in req_languages_list:
        match = re.search(rf'(\w+)\s*\((\w+)\)', lang, re.IGNORECASE)
        if match:
            language, level = match.groups()
            if language.lower() in qualifications.lower():
                user_level = re.search(rf'{language}\s*\((\w+)\)', qualifications, re.IGNORECASE)
                if user_level:
                    user_level = user_level.group(1).lower()
                    if user_level == level.lower():
                        language_score += language_levels[level.lower()]
                    else:
                        # Minuspunkte für niedrigere Stufen
                        language_score += language_levels.get(user_level, 0) - (language_levels[level.lower()] - language_levels.get(user_level, 0))
                else:
                    # Minuspunkte, wenn die Sprache nicht auf dem geforderten Niveau ist
                    language_score -= language_levels[level.lower()] / 2
            else:
                # Zusatzpunkte für jede zusätzliche Sprache
                language_score += language_levels.get(level.lower(), 10) / 2

    score += weight_qualifications * language_score / 100  # Normalisierung des Sprachscores

    # Ausbildung bewerten
    education_levels = {
        'ausbildung': 10,
        'abitur': 20,
        'duales studium': 30,
        'bachelor': 40,
        'master': 50,
        'phd': 60,
        'doktor': 60,
        'quereinstieg': 15,
        'meister': 35,
        'apprenticeship': 10,
        'high school diploma': 20,
        'dual study': 30,
        'bachelor\'s degree': 40,
        'master\'s degree': 50,
        'doctorate': 60,
        'career change': 15,
        'master craftsman': 35
    }

    req_education = job_req['education']
    education_score = 0
    for level, points in education_levels.items():
        if level in education.lower():
            education_score = points
            break

    score += weight_education * education_score

    # Ensure the maximum score is 100
    score = min(score, 100)

    return score, job


def extract_info_from_pdf(file):
    text = ""
    with pdfplumber.open(file) as pdf:
        for page in pdf.pages:
            text += page.extract_text()

    # Initialisiere ein Dictionary, um die Informationen zu speichern
    info = {
        "Vorname": "",
        "Nachname": "",
        "Job": "",
        "E-Mail": "",
        "Geburtstag": "",
        "Berufserfahrung": "",
        "Qualifikationen": "",
        "Ausbildung": ""
    }

    # Durchlaufe die Zeilen der Tabelle und speichere die Informationen
    for row in text.split('\n'):
        if "Vorname" in row:
            info["Vorname"] = row.split(':')[1].strip()
        elif "Nachname" in row:
            info["Nachname"] = row.split(':')[1].strip()
        elif "Job" in row:
            info["Job"] = row.split(':')[1].strip()
        elif "E-Mail" in row:
            info["E-Mail"] = row.split(':')[1].strip()
        elif "Geburtstag" in row:
            info["Geburtstag"] = row.split(':')[1].strip()
        elif "Berufserfahrung" in row:
            info["Berufserfahrung"] = row.split(':')[1].strip()
        elif "Qualifikationen" in row:
            if info["Qualifikationen"]:
                info["Qualifikationen"] += ", " + row.split(':')[1].strip()
            else:
                info["Qualifikationen"] = row.split(':')[1].strip()
        elif "Ausbildung" in row:
            info["Ausbildung"] = row.split(':')[1].strip()

    return info

def extract_info_from_pdf(file):
    with pdfplumber.open(file) as pdf:
        first_page = pdf.pages[0]
        table = first_page.extract_table()

    # Initialisiere ein Dictionary, um die Informationen zu speichern
    info = {
        "Vorname": "",
        "Nachname": "",
        "Job": "",
        "E-Mail": "",
        "Geburtstag": "",
        "Berufserfahrung": "",
        "Qualifikationen": "",
        "Ausbildung": ""
    }

    # Durchlaufe die Zeilen der Tabelle und speichere die Informationen
    for row in table:
        if "Vorname" in row[0]:
            info["Vorname"] = row[1]
        elif "Nachname" in row[0]:
            info["Nachname"] = row[1]
        elif "Job" in row[0]:
            info["Job"] = row[1]
        elif "E-Mail" in row[0]:
            info["E-Mail"] = row[1]
        elif "Geburtstag" in row[0]:
            info["Geburtstag"] = row[1]
        elif "Berufserfahrung" in row[0]:
            info["Berufserfahrung"] = row[1]
        elif "Qualifikationen" in row[0]:
            if info["Qualifikationen"]:
                info["Qualifikationen"] += ", " + row[1]
            else:
                info["Qualifikationen"] = row[1]
        elif "Ausbildung" in row[0]:
            info["Ausbildung"] = row[1]

    return info


In [27]:
from flask import Flask, render_template, request, redirect, url_for, session, jsonify, send_from_directory
import os
import pdfplumber
import random
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import sqlite3
import shutil


app = Flask(__name__)
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly", "https://www.googleapis.com/auth/gmail.compose", "https://www.googleapis.com/auth/gmail.send", "https://www.googleapis.com/auth/gmail.modify"]
app.secret_key = 'abc123'

@app.route('/')
def index():
    try:
        return render_template('index.html')
    except Exception as e:
        return str(e)

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        if username == 'tester' and password == 'tester':
            session['logged_in'] = True  # Setze den Login-Status in der Session
            return redirect(url_for('dashboard'))
        else:
            return 'Ungültiger Benutzername oder Passwort'
    return render_template('login.html')

@app.route('/logout')
def logout():
    session.pop('logged_in', None)  # Entferne den Login-Status aus der Session
    return redirect(url_for('login'))

@app.route('/bewerbung')
def bewerbung():
    return render_template('bewerbung.html')

@app.route('/dashboard', methods=['GET', 'POST'])
def dashboard():
    if not session.get('logged_in'):  # Überprüfe den Login-Status
        return redirect(url_for('login'))

    # Verbindung zur Datenbank herstellen
    conn = sqlite3.connect('bewerber.db')
    cursor = conn.cursor()

    # Daten aus der Datenbank abfragen
    cursor.execute('SELECT id, vorname || " " || nachname AS name, score, job AS position FROM employees')
    scores = [{'id': row[0], 'name': row[1], 'score': row[2], 'position': row[3]} for row in cursor.fetchall()]

    # Einzigartige Job-Positionen aus der Datenbank abfragen
    cursor.execute('SELECT DISTINCT job FROM employees')
    jobs = [row[0] for row in cursor.fetchall()]

    # Verbindung schließen
    conn.close()

    # 'Alle' zur Liste der Positionen hinzufügen
    positions = ['Alle'] + jobs
    
    selected_position = request.form.get('position', 'Alle')
    if selected_position == 'Alle':
        filtered_scores = scores
    else:
        filtered_scores = [score for score in scores if score['position'] == selected_position]
    
    return render_template('dashboard.html', scores=filtered_scores, positions=positions, selected_position=selected_position)

@app.route('/update_scores', methods=['POST'])
def update_scores():
    selected_position = request.json.get('selected_position', 'Alle')

    # Verbindung zur Bewerber-Datenbank herstellen
    conn = sqlite3.connect('bewerber.db')
    cursor = conn.cursor()

    # Daten aus der Datenbank abfragen
    cursor.execute('SELECT id, vorname || " " || nachname AS name, score, job AS position FROM employees')
    scores = [{'id': row[0], 'name': row[1], 'score': row[2], 'position': row[3]} for row in cursor.fetchall()]

    # PDF-Dateien aus dem Ordner lesen und Informationen extrahieren
    pdf_folder = 'attachments'
    processed_folder = 'Bearbeitete_Dokumente'
    os.makedirs(processed_folder, exist_ok=True)

    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            file_path = os.path.join(pdf_folder, filename)
            info = extract_info_from_pdf(file_path)

            # Beispielhafte Einfügung in die Datenbank (hier nur für Demonstrationszwecke)
            score, job = calculate_score(info["Job"], info["Berufserfahrung"], info["Qualifikationen"], info["Ausbildung"])
            cursor.execute('INSERT INTO employees (job, vorname, nachname, email, geburtstag, berufserfahrung, qualifikation, ausbildung, score) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)',
                           (info["Job"], info["Vorname"], info["Nachname"], info["E-Mail"], info["Geburtstag"], info["Berufserfahrung"], info["Qualifikationen"], info["Ausbildung"], score))

            # PDF-Datei in den Ordner Bearbeitete_Dokumente verschieben
            shutil.move(file_path, os.path.join(processed_folder, filename))

    # Verbindung schließen
    conn.commit()
    conn.close()

    if selected_position != 'Alle':
        scores = [score for score in scores if score['position'] == selected_position]

    # Erfolgsmeldung hinzufügen
    response = {
        'message': 'Aktualisierung erfolgreich!',
        'scores': scores
    }

    return jsonify(response)

@app.route('/delete/<int:id>', methods=['POST'])
def delete(id):
    # Verbindung zur Datenbank herstellen
    conn = sqlite3.connect('bewerber.db')
    cursor = conn.cursor()

    # Eintrag löschen
    cursor.execute('DELETE FROM employees WHERE id = ?', (id,))
    conn.commit()

    # Verbindung schließen
    conn.close()

    return redirect(url_for('dashboard'))


@app.route('/api/extract_resume', methods=['POST'])
def extract_resume():
    file = request.files['resume']
    info = extract_info_from_pdf(file)

    result = {
        'firstname': info["Vorname"],
        'lastname': info["Nachname"],
        'birthdate': info["Geburtstag"],
        'job': info["Job"],
        'email': info["E-Mail"],
        'experience': info["Berufserfahrung"],
        'qualifications': info["Qualifikationen"],
        'education': info["Ausbildung"]
    }

    return jsonify(result)

@app.route('/api/fetch_mails', methods=['GET'])
def fetch_mails():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("gmail", "v1", credentials=creds)
        messages = get_unread_emails_with_subject(service, "me", "Bewerbung")
        store_dir = "attachments"
        if not os.path.exists(store_dir):
            os.makedirs(store_dir)
        for msg in messages:
            get_attachments(service, "me", msg['id'], store_dir)
            mark_as_read(service, "me", msg['id'])
        return jsonify({"status": "success", "message": "Mails wurden abgerufen und gespeichert."})
    except HttpError as error:
        return jsonify({"status": "error", "message": str(error)})

@app.route('/submit_application', methods=['POST'])
def submit_application():
    job = request.form.get('job')
    firstname = request.form.get('firstname')
    lastname = request.form.get('lastname')
    email = request.form.get('email')
    birthdate = request.form.get('birthdate')
    experience = request.form.get('experience')
    qualifications = request.form.get('qualifications')
    education = request.form.get('education')
    file = request.files['resume']

    # Score berechnen
    score, job = calculate_score(job, experience, qualifications, education)

    # Verbindung zur Bewerber-Datenbank herstellen
    conn = conn = sqlite3.connect('bewerber.db')
    cursor = conn.cursor()

    # Daten in die Datenbank einfügen
    cursor.execute('''
        INSERT INTO employees (job, vorname, nachname, email, geburtstag, berufserfahrung, qualifikation, ausbildung, score)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (job, firstname, lastname, email, birthdate, experience, qualifications, education, score))
    conn.commit()
    conn.close()

    processed_folder = 'Bearbeitete_Dokumente'
    os.makedirs(processed_folder, exist_ok=True)
    
    filename = file.filename
    file_path = os.path.join(processed_folder, filename)
    base, extension = os.path.splitext(filename)
    counter = 1
    while os.path.exists(file_path):
        new_filename = f"{base}_{counter}{extension}"
        file_path = os.path.join(processed_folder, new_filename)
        counter += 1

    file.save(file_path)

    return jsonify({'status': 'success', 'message': 'Daten erfolgreich eingetragen'})

@app.route('/get_email/<int:id>', methods=['GET'])
def get_email(id):
    # Verbindung zur Bewerber-Datenbank herstellen
    conn1 = sqlite3.connect('bewerber.db')
    cursor1 = conn1.cursor()

    # Informationen basierend auf der ID abfragen
    cursor1.execute('''
        SELECT job, vorname, nachname, email, geburtstag, berufserfahrung, qualifikation, ausbildung, score 
        FROM employees 
        WHERE id = ?
    ''', (id,))
    data1 = cursor1.fetchone()

    # Verbindung zur Bewerber-Datenbank schließen
    conn1.close()

    if data1:
        # Verbindung zur Job-Anforderungen-Datenbank herstellen
        conn2 = sqlite3.connect('job_requirements.db')
        conn2.row_factory = sqlite3.Row
        cursor2 = conn2.cursor()

        # Job-Anforderungen basierend auf dem Jobnamen abfragen
        cursor2.execute('''
            SELECT jobname, experience, qualifications, education, location, weight_experience, weight_qualifications, weight_education 
            FROM job_requirements 
            WHERE jobname = ?
        ''', (data1[0],))
        data2 = cursor2.fetchone()

        # Verbindung zur Job-Anforderungen-Datenbank schließen
        conn2.close()

        if data2:
            return jsonify({
                'job': data1[0],
                'vorname': data1[1],
                'nachname': data1[2],
                'email': data1[3],
                'geburtstag': data1[4],
                'berufserfahrung': data1[5],
                'qualifikation': data1[6],
                'ausbildung': data1[7],
                'score': data1[8],
                'jobname': data2['jobname'],
                'experience': data2['experience'],
                'qualifications': data2['qualifications'],
                'education': data2['education'],
                'location': data2['location'],
                'weight_experience': data2['weight_experience'],
                'weight_qualifications': data2['weight_qualifications'],
                'weight_education': data2['weight_education']
            })
        else:
            return jsonify({'error': 'Job-Anforderungen nicht gefunden'}), 404
    else:
        return jsonify({'error': 'Daten nicht gefunden'}), 404



    
@app.route('/manage_jobs')
def manage_jobs():
    if not session.get('logged_in'):  # Überprüfe den Login-Status
        return redirect(url_for('login'))
    conn = get_db_connection()
    jobs = conn.execute('SELECT * FROM job_requirements').fetchall()
    conn.close()
    return render_template('jobverwaltung.html', jobs=jobs)

@app.route('/create_job', methods=['POST'])
def create_job():
    jobname = request.form['jobname']
    experience = request.form['experience']
    qualifications = request.form['qualifications']
    education = request.form['education']
    location = request.form['location']
    weight_experience = float(request.form['weight_experience'])
    weight_qualifications = float(request.form['weight_qualifications'])
    weight_education = float(request.form['weight_education'])

    conn = get_db_connection()
    conn.execute('''
        INSERT INTO job_requirements (jobname, experience, qualifications, education, location, weight_experience, weight_qualifications, weight_education)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (jobname, experience, qualifications, education, location, weight_experience, weight_qualifications, weight_education))
    conn.commit()
    job_id = conn.execute('SELECT last_insert_rowid()').fetchone()[0]
    conn.close()
    return jsonify({'status': 'success', 'message': 'Job erfolgreich erstellt!', 'job': {'id': job_id, 'jobname': jobname}})

@app.route('/job_details/<int:job_id>')
def job_details(job_id):
    conn = get_db_connection()
    job = conn.execute('SELECT * FROM job_requirements WHERE id = ?', (job_id,)).fetchone()
    conn.close()
    if job is None:
        return jsonify({'status': 'error', 'message': 'Job nicht gefunden!'})
    return jsonify({
        'jobname': job['jobname'],
        'experience': job['experience'],
        'qualifications': job['qualifications'],
        'education': job['education'],
        'location': job['location'],
        'weight_experience': job['weight_experience'],
        'weight_qualifications': job['weight_qualifications'],
        'weight_education': job['weight_education']
    })

@app.route('/update_job/<int:job_id>', methods=['POST'])
def update_job(job_id):
    jobname = request.form['jobname']
    experience = request.form['experience']
    qualifications = request.form['qualifications']
    education = request.form['education']
    location = request.form['location']
    weight_experience = float(request.form['weight_experience'])
    weight_qualifications = float(request.form['weight_qualifications'])
    weight_education = float(request.form['weight_education'])

    conn = get_db_connection()
    conn.execute('''
        UPDATE job_requirements
        SET jobname = ?, experience = ?, qualifications = ?, education = ?, location = ?, weight_experience = ?, weight_qualifications = ?, weight_education = ?
        WHERE id = ?
    ''', (jobname, experience, qualifications, education, location, weight_experience, weight_qualifications, weight_education, job_id))
    conn.commit()
    conn.close()
    return jsonify({'status': 'success', 'message': 'Job erfolgreich aktualisiert!'})

@app.route('/delete_job/<int:job_id>', methods=['DELETE'])
def delete_job(job_id):
    conn = get_db_connection()
    conn.execute('DELETE FROM job_requirements WHERE id = ?', (job_id,))
    conn.commit()
    conn.close()
    return jsonify({'status': 'success', 'message': 'Job erfolgreich gelöscht!'})

@app.route('/stellenanzeigen')
def stellenanzeigen():
    conn = get_db_connection()
    jobs = conn.execute('SELECT * FROM job_requirements').fetchall()
    conn.close()
    return render_template('stellenanzeigen.html', jobs=jobs)

@app.route('/job_details_popup/<int:job_id>')
def job_details_popup(job_id):
    conn = get_db_connection()
    job = conn.execute('SELECT * FROM job_requirements WHERE id = ?', (job_id,)).fetchone()
    conn.close()
    if job is None:
        return jsonify({'status': 'error', 'message': 'Job nicht gefunden!'})
    return jsonify({
        'jobname': job['jobname'],
        'experience': job['experience'],
        'qualifications': job['qualifications'],
        'education': job['education'],
        'location': job['location']
    })

@app.route('/download_template')
def download_template():
    return send_from_directory(directory='.', path="Vorlage_Bewerbung.dotx", as_attachment=True)


    
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Oct/2024 15:18:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 15:18:58] "GET /static/Logo.jpg HTTP/1.1" 304 -
127.0.0.1 - - [20/Oct/2024 15:18:58] "GET /static/background.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [20/Oct/2024 15:18:58] "GET /static/background.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [20/Oct/2024 15:18:58] "GET /static/background.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [20/Oct/2024 15:18:58] "GET /static/background.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [20/Oct/2024 15:19:36] "GET /stellenanzeigen HTTP/1.1" 500 -
Traceback (most recent call last):
  File "c:\Users\bachm\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 1498, in __call__
    return self.wsgi_app(environ, start_response)
           ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bachm\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 1476, in wsgi_app
    response = self.handle_exception